In [2]:
from pathlib import Path
import os
from PIL import Image
import torch
import torchvision.transforms.functional as tf
from splat3d.utils.loss_utils import ssim
from splat3d.lpipsPyTorch import lpips
import json
from tqdm import tqdm
from splat3d.utils.image_utils import psnr

In [6]:
def readImages(renders_dir, gt_dir, gt_rotate=False):
    renders = []
    gts = []
    image_names = []
    for fname in os.listdir(renders_dir):
        render = Image.open(renders_dir / fname)
        gt = Image.open(gt_dir / fname)
        if gt_rotate:
            gt = gt.rotate(180)
        renders.append(tf.to_tensor(render).unsqueeze(0)[:, :3, :, :].cuda())
        gts.append(tf.to_tensor(gt).unsqueeze(0)[:, :3, :, :].cuda())
        image_names.append(fname)
    return renders, gts, image_names

def evaluate(test_dir, gt_name = "gt", renders_name = "renders", method_name = "method", gt_rotate=False):

    full_dict = {}
    per_view_dict = {}
    full_dict_polytopeonly = {}
    per_view_dict_polytopeonly = {}
    print("")

    method = method_name

    full_dict[method] = {}
    per_view_dict[method] = {}
    full_dict_polytopeonly[method] = {}
    per_view_dict_polytopeonly[method] = {}

    method_dir = Path(test_dir)
    gt_dir = method_dir / gt_name
    renders_dir = method_dir/renders_name
    renders, gts, image_names = readImages(renders_dir, gt_dir)

    ssims = []
    psnrs = []
    lpipss = []

    for idx in tqdm(range(len(renders)), desc="Metric evaluation progress"):
        ssims.append(ssim(renders[idx], gts[idx]))
        psnrs.append(psnr(renders[idx], gts[idx]))
        lpipss.append(lpips(renders[idx], gts[idx], net_type='vgg'))

    print("  SSIM : {:>12.7f}".format(torch.tensor(ssims).mean(), ".5"))
    print("  PSNR : {:>12.7f}".format(torch.tensor(psnrs).mean(), ".5"))
    print("  LPIPS: {:>12.7f}".format(torch.tensor(lpipss).mean(), ".5"))
    print("")

    full_dict[method].update({"SSIM": torch.tensor(ssims).mean().item(),
                                            "PSNR": torch.tensor(psnrs).mean().item(),
                                            "LPIPS": torch.tensor(lpipss).mean().item()})
    per_view_dict[method].update({"SSIM": {name: ssim for ssim, name in zip(torch.tensor(ssims).tolist(), image_names)},
                                                "PSNR": {name: psnr for psnr, name in zip(torch.tensor(psnrs).tolist(), image_names)},
                                                "LPIPS": {name: lp for lp, name in zip(torch.tensor(lpipss).tolist(), image_names)}})

    with open(test_dir + "/results.json", 'w') as fp:
        json.dump(full_dict, fp, indent=True)
    with open(test_dir + "/per_view.json", 'w') as fp:
        json.dump(per_view_dict, fp, indent=True)

# NeRF

In [4]:
evaluate("models/nerf/Case_I/renderonly_test_049999", gt_name="gt", renders_name="estim", method_name="nerf")
evaluate("models/nerf/Case_II/renderonly_test_049999", gt_name="gt", renders_name="estim", method_name="nerf")
evaluate("models/nerf/Case_III/renderonly_test_049999", gt_name="gt", renders_name="estim", method_name="nerf")
evaluate("models/nerf/Case_IV/renderonly_test_049999", gt_name="gt", renders_name="estim", method_name="nerf")

Metric evaluation progress: 100%|██████████| 2/2 [00:01<00:00,  1.13it/s]


  SSIM :    0.2154723
  PSNR :   14.1862488
  LPIPS:    0.6419058




Metric evaluation progress: 100%|██████████| 4/4 [00:03<00:00,  1.29it/s]


  SSIM :    0.2602040
  PSNR :   11.9458132
  LPIPS:    0.6269214




Metric evaluation progress: 100%|██████████| 10/10 [00:07<00:00,  1.27it/s]


  SSIM :    0.8299262
  PSNR :   14.5863972
  LPIPS:    0.1775394




Metric evaluation progress: 100%|██████████| 3/3 [00:02<00:00,  1.28it/s]

  SSIM :    0.4839024
  PSNR :   12.7605028
  LPIPS:    0.4655773



# D-NeRF

In [18]:
evaluate("models/dnerf/Case_I/renderonly_test_049999", gt_name="gt", renders_name="estim", method_name="dnerf")
evaluate("models/dnerf/Case_II/renderonly_test_049999", gt_name="gt", renders_name="estim", method_name="dnerf")
evaluate("models/dnerf/Case_III/renderonly_test_049999", gt_name="gt", renders_name="estim", method_name="dnerf")
evaluate("models/dnerf/Case_IV/renderonly_test_049999", gt_name="gt", renders_name="estim", method_name="dnerf")

Metric evaluation progress:   0%|          | 0/2 [00:00<?, ?it/s]

Metric evaluation progress: 100%|██████████| 2/2 [00:01<00:00,  1.01it/s]


  SSIM :    0.3782941
  PSNR :   16.7554207
  LPIPS:    0.5417533




Metric evaluation progress: 100%|██████████| 4/4 [00:03<00:00,  1.02it/s]


  SSIM :    0.0337398
  PSNR :    9.2241917
  LPIPS:    0.5877476




Metric evaluation progress: 100%|██████████| 6/6 [00:06<00:00,  1.00s/it]


  SSIM :    0.8156426
  PSNR :   16.6558895
  LPIPS:    0.1853004




Metric evaluation progress: 100%|██████████| 3/3 [00:03<00:00,  1.00s/it]

  SSIM :    0.6183863
  PSNR :   19.6067581
  LPIPS:    0.3281882



# Instant NeRF

In [7]:
evaluate("models/ngp/Case_I", gt_name="gt", renders_name="estim", method_name="nerf", gt_rotate=True)
evaluate("models/ngp/Case_II", gt_name="gt", renders_name="estim", method_name="nerf", gt_rotate=True)
evaluate("models/ngp/Case_III", gt_name="gt", renders_name="estim", method_name="nerf", gt_rotate=True)
evaluate("models/ngp/Case_IV", gt_name="gt", renders_name="estim", method_name="nerf", gt_rotate=True)

Metric evaluation progress:   0%|          | 0/2 [00:00<?, ?it/s]

Metric evaluation progress: 100%|██████████| 2/2 [00:01<00:00,  1.27it/s]


  SSIM :    0.5149162
  PSNR :   16.4692898
  LPIPS:    0.4374050




Metric evaluation progress: 100%|██████████| 3/3 [00:02<00:00,  1.27it/s]


  SSIM :    0.4728819
  PSNR :   14.7125664
  LPIPS:    0.4440168




Metric evaluation progress: 100%|██████████| 10/10 [00:08<00:00,  1.21it/s]


  SSIM :    0.8570850
  PSNR :   17.5529118
  LPIPS:    0.1276675




Metric evaluation progress: 100%|██████████| 2/2 [00:01<00:00,  1.24it/s]

  SSIM :    0.8568976
  PSNR :   19.9869423
  LPIPS:    0.1055752



# Splat3D

In [16]:
!python splat3d/metrics.py -m models/splat/Case_I/
!python splat3d/metrics.py -m models/splat/Case_II/
!python splat3d/metrics.py -m models/splat/Case_III/
!python splat3d/metrics.py -m models/splat/Case_IV/


Scene: models/splat/Case_I/
Method: ours_30000
Metric evaluation progress: 100%|█████████████████| 2/2 [00:02<00:00,  1.10s/it]
  SSIM :    0.3463362
  PSNR :   14.7913246
  LPIPS:    0.5164169


Scene: models/splat/Case_II/
Method: ours_30000
Metric evaluation progress: 100%|█████████████████| 4/4 [00:04<00:00,  1.04s/it]
  SSIM :    0.3867278
  PSNR :   13.4230061
  LPIPS:    0.5141455


Scene: models/splat/Case_III/
Method: ours_30000
Metric evaluation progress: 100%|█████████████████| 6/6 [00:06<00:00,  1.07s/it]
  SSIM :    0.6834939
  PSNR :   21.0664902
  LPIPS:    0.1267552


Scene: models/splat/Case_IV/
Method: ours_30000
Metric evaluation progress: 100%|█████████████████| 3/3 [00:03<00:00,  1.07s/it]
  SSIM :    0.5929787
  PSNR :   20.3123474
  LPIPS:    0.1856166



# D-Splat3D

In [2]:
!python splat3d/metrics.py -m output/dsplat/Case_I
!python splat3d/metrics.py -m output/dsplat/Case_II
!python splat3d/metrics.py -m output/dsplat/Case_III
!python splat3d/metrics.py -m output/dsplat/Case_IV


Scene: output/dsplat/Case_I
Method: ours_14000
Metric evaluation progress: 100%|█████████████████| 2/2 [00:01<00:00,  1.14it/s]
  SSIM :    0.3956457
  PSNR :   15.0140953
  LPIPS:    0.5365519


Scene: output/dsplat/Case_II
Method: ours_14000
Metric evaluation progress: 100%|█████████████████| 3/3 [00:02<00:00,  1.17it/s]
  SSIM :    0.4111283
  PSNR :   11.8850985
  LPIPS:    0.5601883


Scene: output/dsplat/Case_III
Method: ours_14000
Metric evaluation progress: 100%|███████████████| 10/10 [00:08<00:00,  1.20it/s]
  SSIM :    0.3653284
  PSNR :   14.7108517
  LPIPS:    0.1363462


Scene: output/dsplat/Case_IV
Method: ours_14000
Metric evaluation progress: 100%|█████████████████| 2/2 [00:01<00:00,  1.13it/s]
  SSIM :    0.7618935
  PSNR :   17.1552429
  LPIPS:    0.1889512



# 4DGS

In [2]:
!python splat3d/metrics.py -m models/4dgs/Case_I
!python splat3d/metrics.py -m models/4dgs/Case_II
!python splat3d/metrics.py -m models/4dgs/Case_III
!python splat3d/metrics.py -m models/4dgs/Case_IV


Scene: models/4dgs/Case_I
Method: ours_14000
Metric evaluation progress: 100%|█████████████████| 2/2 [00:01<00:00,  1.10it/s]
  SSIM :    0.4049577
  PSNR :   14.4832954
  LPIPS:    0.5258497


Scene: models/4dgs/Case_II
Method: ours_14000
Metric evaluation progress: 100%|█████████████████| 3/3 [00:02<00:00,  1.12it/s]
  SSIM :    0.4379860
  PSNR :   11.8324623
  LPIPS:    0.5563418


Scene: models/4dgs/Case_III
Method: ours_14000
Metric evaluation progress: 100%|███████████████| 10/10 [00:08<00:00,  1.16it/s]
  SSIM :    0.3653284
  PSNR :   14.7108517
  LPIPS:    0.1363463


Scene: models/4dgs/Case_IV
Method: ours_14000
Metric evaluation progress: 100%|█████████████████| 2/2 [00:01<00:00,  1.12it/s]
  SSIM :    0.7143956
  PSNR :   13.2914219
  LPIPS:    0.2223410

